In [5]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt

mkdir: cannot create directory ‘data/’: File exists
--2022-04-30 02:33:46--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  19.5MB/s    in 7m 47s  

2022-04-30 02:41:34 (13.8 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-04-30 02:42:26--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving

In [29]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.MobileNetV2(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [30]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))


1%, Top 1:0.684 Top 5:0.9
2%, Top 1:0.672 Top 5:0.888
3%, Top 1:0.682 Top 5:0.8866666666666667
4%, Top 1:0.677 Top 5:0.8835
5%, Top 1:0.6824 Top 5:0.884
6%, Top 1:0.6886666666666666 Top 5:0.8833333333333333
7%, Top 1:0.6865714285714286 Top 5:0.8862857142857142
8%, Top 1:0.68575 Top 5:0.88375
9%, Top 1:0.688 Top 5:0.882
10%, Top 1:0.6898 Top 5:0.882
11%, Top 1:0.6894545454545454 Top 5:0.8812727272727273
12%, Top 1:0.6891666666666667 Top 5:0.8828333333333334
13%, Top 1:0.6887692307692308 Top 5:0.8829230769230769
14%, Top 1:0.6891428571428572 Top 5:0.8835714285714286
15%, Top 1:0.6890666666666667 Top 5:0.8841333333333333
16%, Top 1:0.6895 Top 5:0.88475
17%, Top 1:0.6896470588235294 Top 5:0.8847058823529412
18%, Top 1:0.6886666666666666 Top 5:0.884
19%, Top 1:0.6893684210526316 Top 5:0.8841052631578947
20%, Top 1:0.6886 Top 5:0.8835
21%, Top 1:0.6884761904761905 Top 5:0.8845714285714286
22%, Top 1:0.6885454545454546 Top 5:0.884
23%, Top 1:0.6893913043478261 Top 5:0.8832173913043478
24%, To

In [31]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:68.052%
Top 5 Accuracy:88.396%
